In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<h1> Feature Extraction </h1>

<p>
<b>lags</b> -> [28,35,42,49,56,63,70,77] (From ACF and PACF plots) <br>
<b>rolling</b> ->lags = [7,14,28,49] with shift 28<br>
<b>weekday/weekend</b> -> 0/1<br>
<b>quarter of year</b> -> [1,2,3,4]<br>
<b>id wise sale mean and standard deviation</b><br>
<b>id wise price mean and standard deviation</b><br>
<b>expanding sold mean with shift 28</b><br> 
</p>

In [3]:
d = {'sale'           :  np.int16,      
     'snap_CA'        :  np.int8,   
     'snap_TX'        :  np.int8,   
     'snap_WI'        :  np.int8,   
     'sell_price'     :  np.float32,
     'wm_yr_wk'       :  np.int16,
     'year'           : np.int8,
     'event_name_1'   :  np.int8,   
     'event_type_1'   :  np.int8,   
     'event_name_2'   :  np.int8,   
     'event_type_2'   :  np.int8,
     'd'              : np.int16,
     'day'            : np.int8,
     'quarter'        : np.int8
    }

In [4]:
df = pd.read_csv('/content/drive/MyDrive/CS/merged_df.csv',dtype=d)

In [5]:
from sklearn.preprocessing import LabelEncoder
#label encoding all categorical value
le = {}
for ele in df.columns:
    if df[ele].dtype.name == 'object':
        labelencoder=LabelEncoder()
        df[ele] = df[ele].astype(str)
        df[ele] = labelencoder.fit_transform(df[ele]).astype(np.int16)
        le[ele] = dict(zip(labelencoder.classes_, labelencoder.transform(labelencoder.classes_)))

In [24]:
import pickle
with open('/content/drive/MyDrive/CS/label_encoding.pickle','wb') as f: #saving label encoded into a pickle file as dictionary
    pickle.dump(le,f)

In [7]:
groupby_df = df.groupby(['id'],as_index=False)

In [8]:
#we don;t have lags from 0 to 28 for d_1941 to d_1969 so using lags from 28
lags = [28,35,42,49,56,63,70,77]  #from autocorrelation plot
#https://stackoverflow.com/questions/37231844/pandas-creating-a-lagged-column-with-grouped-data
for ele in lags:
    df['lag_'+str(ele)] = groupby_df['sale'].shift(ele) #https://stackoverflow.com/questions/33907537/groupby-and-lag-all-columns-of-a-dataframe

In [9]:
lags = [7,14,28,49]  
for ele in lags:
    df['roll_mean_'+str(ele)] = groupby_df['sale'].transform(lambda x: x.shift(28).rolling(ele).mean()) #https://stackoverflow.com/questions/53339021/python-pandas-calculate-moving-average-within-group
    df['roll_std_'+str(ele)] = groupby_df['sale'].transform(lambda x: x.shift(28).rolling(ele).std())

In [10]:
df['expanding_mean_item']=groupby_df['sale'].transform(lambda x: x.shift(28).expanding().mean()) #expanding mean

#mean and std of id sold
df['mean_id_sold'] = groupby_df['sale'].transform('mean').astype(np.float16)
df['std_id_sold'] = groupby_df['sale'].transform('std').astype(np.float16)

#mean and std of price
df['mean_id_price'] = groupby_df['sell_price'].transform('mean').astype(np.float16)
df['std_id_price'] = groupby_df['sell_price'].transform('std').astype(np.float16)

In [11]:
del groupby_df

In [12]:
df.dropna(how='any',inplace=True) #removing NaN values produced due to lags

In [13]:
df.drop(columns = ['wm_yr_wk'],inplace=True)
#Downcasting for space optimization
df['sale'] = df['sale'].astype(np.int16)
df['snap_CA'] = df['snap_CA'].astype(np.int8)
df['snap_TX'] = df['snap_TX'].astype(np.int8)
df['snap_WI'] = df['snap_WI'].astype(np.int8)

df['expanding_mean_item'] = df['expanding_mean_item'].astype(np.float16)

df['lag_28'] = df['lag_28'].astype(np.int16)
df['lag_35'] = df['lag_35'].astype(np.int16)
df['lag_42'] = df['lag_42'].astype(np.int16)
df['lag_49'] = df['lag_49'].astype(np.int16)
df['lag_56'] = df['lag_56'].astype(np.int16)
df['lag_63'] = df['lag_63'].astype(np.int16)
df['lag_70'] = df['lag_70'].astype(np.int16)
df['lag_77'] = df['lag_77'].astype(np.int16)

df['roll_mean_7'] = df['roll_mean_7'].astype(np.float16)
df['roll_mean_14'] = df['roll_mean_14'].astype(np.float16)
df['roll_std_7'] = df['roll_std_7'].astype(np.float16)
df['roll_std_14'] = df['roll_std_14'].astype(np.float16)
df['roll_mean_28'] = df['roll_mean_28'].astype(np.float16)
df['roll_mean_49'] = df['roll_mean_49'].astype(np.float16)
df['roll_std_28'] = df['roll_std_28'].astype(np.float16)
df['roll_std_49'] = df['roll_std_49'].astype(np.float16)

In [16]:
df[df['d']>1941].tail(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,year,event_name_1,...,roll_std_14,roll_mean_28,roll_std_28,roll_mean_49,roll_std_49,expanding_mean_item,mean_id_sold,std_id_sold,mean_id_price,std_id_price
60034800,14279,1427,2,0,9,2,1969,0,6,17,...,1.392578,2.000000,1.540039,1.816406,1.481445,1.242188,1.224609,1.532227,2.281250,1.263672
60034801,14289,1428,2,0,9,2,1969,0,6,17,...,1.703125,2.000000,1.763672,2.164062,1.771484,2.714844,2.677734,3.369141,2.023438,0.082458
60034802,14299,1429,2,0,9,2,1969,0,6,17,...,1.610352,1.607422,1.256836,1.673828,1.313477,1.546875,1.524414,1.494141,1.966797,0.072205
60034803,14309,1430,2,0,9,2,1969,0,6,17,...,1.392578,1.071289,1.274414,1.061523,1.375977,0.650391,0.641113,1.109375,2.978516,2.441406
60034804,14319,1431,2,0,9,2,1969,0,6,17,...,1.492188,2.072266,1.513672,2.244141,1.677734,3.154297,3.109375,3.294922,3.837891,0.466309
60034805,14329,1432,2,0,9,2,1969,0,6,17,...,0.841797,0.643066,0.826172,0.489746,0.767090,0.535645,0.528320,1.166016,2.800781,0.171265
60034806,14339,1433,2,0,9,2,1969,0,6,17,...,0.363037,0.285645,0.534668,0.306152,0.547852,0.375000,0.369629,0.811523,2.507812,0.253174
60034807,14349,1434,2,0,9,2,1969,0,6,17,...,1.109375,0.785645,0.916992,0.897949,1.084961,0.893555,0.880859,1.374023,4.117188,0.188477
60034808,14359,1435,2,0,9,2,1969,0,6,17,...,1.650391,1.321289,1.306641,1.204102,1.189453,0.380615,0.375244,0.967773,0.670410,0.639648
60034809,14369,1436,2,0,9,2,1969,0,6,17,...,1.823242,1.250000,1.481445,0.877441,1.317383,0.702637,0.692871,1.500977,0.427734,0.494873


In [23]:
df.to_csv('/content/drive/MyDrive/CS/final_fe2.csv',index=False)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57687080 entries, 2347730 to 60034809
Data columns (total 40 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int16  
 1   item_id              int16  
 2   dept_id              int16  
 3   cat_id               int16  
 4   store_id             int16  
 5   state_id             int16  
 6   d                    int16  
 7   sale                 int16  
 8   year                 int8   
 9   event_name_1         int8   
 10  event_type_1         int8   
 11  event_name_2         int8   
 12  event_type_2         int8   
 13  snap_CA              int8   
 14  snap_TX              int8   
 15  snap_WI              int8   
 16  day                  int8   
 17  quarter              int8   
 18  sell_price           float32
 19  lag_28               int16  
 20  lag_35               int16  
 21  lag_42               int16  
 22  lag_49               int16  
 23  lag_56               int

In [18]:
df.isnull().sum() #checking NaN values again

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
d                      0
sale                   0
year                   0
event_name_1           0
event_type_1           0
event_name_2           0
event_type_2           0
snap_CA                0
snap_TX                0
snap_WI                0
day                    0
quarter                0
sell_price             0
lag_28                 0
lag_35                 0
lag_42                 0
lag_49                 0
lag_56                 0
lag_63                 0
lag_70                 0
lag_77                 0
roll_mean_7            0
roll_std_7             0
roll_mean_14           0
roll_std_14            0
roll_mean_28           0
roll_std_28            0
roll_mean_49           0
roll_std_49            0
expanding_mean_item    0
mean_id_sold           0
std_id_sold            0
mean_id_price          0
std_id_price           0
